In [1]:
import os
import pandas as pd
import numpy as np

#### **Preparing Necessary Dataset and Function**

In [2]:
accident_df = pd.read_csv('C:/Users/Asus/Downloads/COMP20008/datasets/accident.csv', usecols=[
    'ACCIDENT_NO', 'SEVERITY','ACCIDENT_DATE',
    'LIGHT_CONDITION', 'ROAD_GEOMETRY','ROAD_GEOMETRY_DESC','SPEED_ZONE','ACCIDENT_TIME'])

atmospheric_df = pd.read_csv('C:/Users/Asus/Downloads/COMP20008/datasets/atmospheric_cond.csv', usecols=[
    'ACCIDENT_NO','ATMOSPH_COND','ATMOSPH_COND_DESC'])

road_df = pd.read_csv('C:/Users/Asus/Downloads/COMP20008/datasets/road_surface_cond.csv', usecols=[
    'ACCIDENT_NO','SURFACE_COND', 'SURFACE_COND_DESC'])

vehicle_df = pd.read_csv('C:/Users/Asus/Downloads/COMP20008/datasets/filtered_vehicle.csv', usecols=[
    'ACCIDENT_NO','VEHICLE_YEAR_MANUF', 'VEHICLE_TYPE','TOTAL_NO_OCCUPANTS','ROAD_SURFACE_TYPE',
    'TRAFFIC_CONTROL','TRAFFIC_CONTROL_DESC'])

person_df = pd.read_csv('C:/Users/Asus/Downloads/COMP20008/datasets/person.csv', usecols=[
    'ACCIDENT_NO', 'HELMET_BELT_WORN','AGE_GROUP','SEATING_POSITION'])

accident_merged = accident_df.merge(atmospheric_df, on='ACCIDENT_NO', how='left').merge(road_df, on='ACCIDENT_NO', how='left')

*Defining Function*

In [3]:
# Defining custom manual MODE
# Will be used in non-numeric columns
def custom_mode(x):
    m = x.mode()
    return m.iloc[0] if not m.empty else None

# Defining outlier detection function
# Will be used in cleaned dataset
def iqr_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return series[(series < lower_bound) | (series > upper_bound)]

#### **Aggregating Vehicle Dataset**

In [4]:
## Aggregating Vehicle DataFrame

accident_merged["ACCIDENT_YEAR"] = pd.to_datetime(accident_merged["ACCIDENT_DATE"]).dt.year
vehicle_df = vehicle_df.merge(accident_merged[["ACCIDENT_NO", "ACCIDENT_YEAR"]], on="ACCIDENT_NO", how="left")

# Aggregate using MEDIAN for vehicle year manufactured and accident year
med_v_yr = vehicle_df["VEHICLE_YEAR_MANUF"].median()
med_acc_yr = vehicle_df["ACCIDENT_YEAR"].median()

vehicle_df["VEHICLE_YEAR_MANUF"] = vehicle_df["VEHICLE_YEAR_MANUF"].fillna(med_v_yr)
vehicle_df["ACCIDENT_YEAR"] = vehicle_df["ACCIDENT_YEAR"].fillna(med_acc_yr)


# Computing vehicle age from accident year and vehicle year manufactured
vehicle_df["VEHICLE_AGE"] = vehicle_df["ACCIDENT_YEAR"] - vehicle_df["VEHICLE_YEAR_MANUF"]

# Aggregating Vehicle Age and Total Occupant using median and sum
v_agg = vehicle_df.groupby("ACCIDENT_NO").agg({
    "VEHICLE_AGE": "median",
    "TOTAL_NO_OCCUPANTS": "sum"
}).reset_index()

# Aggregating Vehicle Type, Traffic Control, and Road Surface Type using custom MODE
v_type = vehicle_df.groupby("ACCIDENT_NO")["VEHICLE_TYPE"].apply(custom_mode)
trf_control = vehicle_df.groupby("ACCIDENT_NO")["TRAFFIC_CONTROL"].apply(custom_mode)
rd_surf = vehicle_df.groupby("ACCIDENT_NO")["ROAD_SURFACE_TYPE"].apply(custom_mode)

v_agg["VEHICLE_TYPE"] = v_type.values
v_agg["TRAFFIC_CONTROL"] = trf_control.values
v_agg["ROAD_SURFACE_TYPE"] = rd_surf.values

accident_merged = accident_merged.merge(v_agg, on="ACCIDENT_NO", how="left")


#### **Aggregating Person Dataset**

In [5]:
# Replacing NA in SEATING_POSITION with NA_val
# This is to treat actual NA value as valid value
person_df["SEATING_POSITION"] = person_df["SEATING_POSITION"].fillna("NA_val")

# Aggregating age group, seating position and helmet/belt worn using custom MODE
age_g = person_df.groupby("ACCIDENT_NO")["AGE_GROUP"].apply(custom_mode)
seating = person_df.groupby("ACCIDENT_NO")["SEATING_POSITION"].apply(custom_mode)
belt_helm = person_df.groupby("ACCIDENT_NO")["HELMET_BELT_WORN"].apply(custom_mode)

# Aggregating Vehicle Type, Traffic Control, and Road Surface Type using custom MODE
person_agg = pd.DataFrame({
    "AGE_GROUP": age_g,
    "SEATING_POSITION": seating,
    "HELMET_BELT_WORN": belt_helm
}).reset_index()

accident_merged = accident_merged.merge(person_agg, on="ACCIDENT_NO", how="left")


#### **Cleaning Merged Dataset**

*Checking Unknown or NA values of new object type column*

In [6]:
accident_merged['VEHICLE_TYPE'].unique()

array([nan,  1.,  4.,  2.,  3., 72., 71., 61., 62.,  5.,  8., 17.,  9.,
       60., 10., 27., 20., 63., 99., 18.,  7.,  6.])

In [7]:
accident_merged.dtypes

ACCIDENT_NO            object
ACCIDENT_DATE          object
ACCIDENT_TIME          object
LIGHT_CONDITION         int64
ROAD_GEOMETRY           int64
ROAD_GEOMETRY_DESC     object
SEVERITY                int64
SPEED_ZONE              int64
ATMOSPH_COND            int64
ATMOSPH_COND_DESC      object
SURFACE_COND            int64
SURFACE_COND_DESC      object
ACCIDENT_YEAR           int32
VEHICLE_AGE           float64
TOTAL_NO_OCCUPANTS    float64
VEHICLE_TYPE          float64
TRAFFIC_CONTROL       float64
ROAD_SURFACE_TYPE     float64
AGE_GROUP              object
SEATING_POSITION       object
HELMET_BELT_WORN      float64
dtype: object

In [8]:
categoric_col = ['LIGHT_CONDITION','ROAD_GEOMETRY','SPEED_ZONE','ATMOSPH_COND',
                 'SURFACE_COND','VEHICLE_TYPE','TRAFFIC_CONTROL','ROAD_SURFACE_TYPE',
                 'AGE_GROUP','SEATING_POSITION','HELMET_BELT_WORN']

for i in range(len(categoric_col)):
    print('Column: {}'.format(categoric_col[i]))
    print(np.sort(accident_merged[categoric_col[i]].unique()))

Column: LIGHT_CONDITION
[1 2 3 4 5 6 9]
Column: ROAD_GEOMETRY
[1 2 3 4 5 6 7 8 9]
Column: SPEED_ZONE
[ 30  40  50  60  70  75  80  90 100 110 777 888 999]
Column: ATMOSPH_COND
[1 2 3 4 5 6 7 9]
Column: SURFACE_COND
[1 2 3 4 5 9]
Column: VEHICLE_TYPE
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 17. 18. 20. 27. 60. 61. 62. 63.
 71. 72. 99. nan]
Column: TRAFFIC_CONTROL
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 99. nan]
Column: ROAD_SURFACE_TYPE
[ 1.  2.  3.  9. nan]
Column: AGE_GROUP
['0-4' '13-15' '16-17' '18-21' '22-25' '26-29' '30-39' '40-49' '5-12'
 '50-59' '60-64' '65-69' '70+' 'Unknown']
Column: SEATING_POSITION
['CF' 'CR' 'D' 'LF' 'LR' 'NA_val' 'NK' 'OR' 'PL' 'RR']
Column: HELMET_BELT_WORN
[1. 2. 3. 4. 5. 6. 7. 8. 9.]


Based on the Victoria Police Report Documentation, there are some category such as Unknown, Not Available, Not Known that will be dropped so that our analysis and interpretation will be more comprehensive. Such categories are as follows.

In [9]:
# Define filters for each column

invalidLight = 9
invalidRoadGeom = 9
invalidSpeedZone = [777, 888, 999]
invalidAtmCond = 9
invalidSurfCond = 9
invalidVehType = [17, 18, 27, 99]
invalidTraffCont = [15, 99]
invalidRoadSurf = 9
invalidAgeGroup = 'Unknown'
invalidSeating = ["NA_val", "NK"]
invalidHelmetBelt = [8, 9]

In [10]:
# Apply all filters
accident_clean = accident_merged[
    (accident_merged["LIGHT_CONDITION"] != invalidLight) &
    (accident_merged["ROAD_GEOMETRY"] != invalidRoadGeom) &
    (~accident_merged["SPEED_ZONE"].isin(invalidSpeedZone)) &
    (accident_merged["ATMOSPH_COND"] != invalidAtmCond) &
    (accident_merged["SURFACE_COND"] != invalidSurfCond) &
    (~accident_merged["VEHICLE_TYPE"].isin(invalidVehType)) &
    (~accident_merged["TRAFFIC_CONTROL"].isin(invalidTraffCont)) &
    (accident_merged["ROAD_SURFACE_TYPE"]!= invalidRoadSurf) &
    (accident_merged["AGE_GROUP"] != invalidAgeGroup) &
    (~accident_merged["SEATING_POSITION"].isin(invalidSeating)) &
    (~accident_merged["HELMET_BELT_WORN"].isin(invalidHelmetBelt)) &
    accident_merged["VEHICLE_TYPE"].notna() &
    accident_merged["TRAFFIC_CONTROL"].notna() &
    accident_merged["ROAD_SURFACE_TYPE"].notna()
   ]

In [11]:
categoric_col = ['LIGHT_CONDITION','ROAD_GEOMETRY','SPEED_ZONE','ATMOSPH_COND',
                 'SURFACE_COND','VEHICLE_TYPE','TRAFFIC_CONTROL','ROAD_SURFACE_TYPE',
                 'AGE_GROUP','SEATING_POSITION','HELMET_BELT_WORN']

for i in range(len(categoric_col)):
    print('Column: {}'.format(categoric_col[i]))
    print(np.sort(accident_clean[categoric_col[i]].unique()))

Column: LIGHT_CONDITION
[1 2 3 4 5 6]
Column: ROAD_GEOMETRY
[1 2 3 4 5 6 7 8]
Column: SPEED_ZONE
[ 30  40  50  60  70  75  80  90 100 110]
Column: ATMOSPH_COND
[1 2 3 4 5 6 7]
Column: SURFACE_COND
[1 2 3 4 5]
Column: VEHICLE_TYPE
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 20. 60. 61. 62. 63. 71. 72.]
Column: TRAFFIC_CONTROL
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
Column: ROAD_SURFACE_TYPE
[1. 2. 3.]
Column: AGE_GROUP
['0-4' '13-15' '16-17' '18-21' '22-25' '26-29' '30-39' '40-49' '5-12'
 '50-59' '60-64' '65-69' '70+']
Column: SEATING_POSITION
['CF' 'CR' 'D' 'LF' 'LR' 'OR' 'PL' 'RR']
Column: HELMET_BELT_WORN
[1. 2. 3. 4. 5. 6. 7.]


In [12]:
accident_clean.shape

(102302, 21)

In [13]:
accident_clean = accident_clean.copy()

#### **Making New Mapping in Cleaned Dataset**

*Making categorical features*

In [15]:
# Reversing Severity Mapping
severity_ord = {4: 0, 3: 1, 2: 2, 1: 3}
accident_clean.loc[:,'SEVERITY_ORD'] = accident_clean['SEVERITY'].map(severity_ord)

# Ordinal Age Variable
age_ord = {'0-4' : 0, '5-12' : 0,
           '13-15' : 1, '16-17' : 1, '18-21' : 1,
           '22-25' : 2, '26-29' : 2,
           '30-39': 3,
           '40-49': 4,
           '50-59': 5,
           '60-64' : 6, '65-69': 6,
           '70+': 7}
accident_clean.loc[:, 'AGE_ORD'] = accident_clean['AGE_GROUP'].map(age_ord)

# Categorical Seating Map Variable
seat_cat = { "D": 1, "LF": 2, "CF": 3,
    "OR": 4, "CR": 5, "LR": 6, "RR": 7, "PL": 8}
accident_clean.loc[:, "SEAT_CATEGORY"] = accident_clean["SEATING_POSITION"].map(seat_cat)

# Vehicle Type Category Remapping
veh_cat = { 10: 1, 20: 1,                   # 2-wheelers
           1: 2, 2: 2, 3: 2, 4: 2, 5: 2,    # 4-wheelers 
           8: 3, 9: 3,                      # Buses
           6: 4, 7: 4, 60: 4, 61: 4,        # Heavy vehicle
           62: 4, 63: 4, 71: 4, 72: 4}
accident_clean.loc[:, "VEHICLE_TYPE_CAT"] = accident_clean["VEHICLE_TYPE"].map(veh_cat)

# Mapping Accident Time
accident_clean['ACCIDENT_TIME']=pd.to_datetime(accident_clean['ACCIDENT_TIME'],format='%H:%M:%S').dt.time

accident_clean['TIME_OF_DAY'] = ''
accident_clean.loc[(accident_clean['ACCIDENT_TIME'] >= pd.to_datetime("06:00:00").time()) & (accident_clean['ACCIDENT_TIME'] <= pd.to_datetime("11:59:59").time()), 'TIME_OF_DAY'] = 1
accident_clean.loc[(accident_clean['ACCIDENT_TIME'] >= pd.to_datetime("12:00:00").time()) & (accident_clean['ACCIDENT_TIME'] <= pd.to_datetime("17:59:59").time()), 'TIME_OF_DAY'] = 2
accident_clean.loc[(accident_clean['ACCIDENT_TIME'] >= pd.to_datetime("18:00:00").time()) & (accident_clean['ACCIDENT_TIME'] <= pd.to_datetime("23:59:59").time()), 'TIME_OF_DAY'] = 3
accident_clean.loc[(accident_clean['ACCIDENT_TIME'] >= pd.to_datetime("00:00:00").time()) & (accident_clean['ACCIDENT_TIME'] <= pd.to_datetime("05:59:59").time()), 'TIME_OF_DAY'] = 4

*Making binary from each categorical features*

In [16]:
# Mapping Binary Severity
severity_bin = {0: 0, 1: 0, 2: 0, 3: 1}
accident_clean["SEVERITY_BINARY"] = accident_clean["SEVERITY_ORD"].map(severity_bin)

# Binary Road Surface
surf_bin = {"Dry": 0, "Wet": 1, "Icy": 1, "Muddy": 1, "Snowy": 1}
accident_clean.loc[:, "SURFACE_COND_BINARY"] = accident_clean["SURFACE_COND_DESC"].map(surf_bin)

# Binary Atmospheric Condition
atm_bin = { "Clear": 0, "Raining": 1, "Strong winds": 1,
    "Fog": 1, "Dust": 1, "Smoke": 1, "Snowing": 1}
accident_clean.loc[:, "ATMOSPH_COND_BINARY"] = accident_clean["ATMOSPH_COND_DESC"].map(atm_bin)

# Binary Seating Map Variable
seat_bin = {"OR" : 0, "CR" : 0, "LR" : 0, "RR" : 0, "PL": 0,
            "D" : 1, "LF" : 1, "CF": 1}
accident_clean.loc[:, 'SEAT_BINARY'] = accident_clean['SEATING_POSITION'].map(seat_bin)

# Binary Seatbelt / Helmet
belthelm_bin = {1: 0, 3: 0, 6: 0,
                2: 1, 4: 1, 5: 1, 7: 1}
accident_clean.loc[:, 'HELMET_BELT_BINARY'] = accident_clean['HELMET_BELT_WORN'].map(belthelm_bin)

# Binary Traffic Control
traffic_bin = { 1: 0, 2: 0, 4: 0, 5: 0, 6: 0, 7: 0, 9: 0, 10: 0, 11: 0, 12: 0, 14: 0,
               0: 1, 3: 1, 8: 1, 13: 1}
accident_clean['TRAFFIC_CONTROL_BINARY'] = accident_clean['TRAFFIC_CONTROL'].map(traffic_bin)

# Binary Intersection
geometry_bin = {5: 0, 6: 0, 7: 0, 8: 0,
                1: 1, 2: 1, 3: 1, 4: 1}
accident_clean['ROAD_GEOMETRY_BINARY'] = accident_clean['ROAD_GEOMETRY'].map(geometry_bin)

# Binary Light
light_bin = {1: 0, 2: 0, 3: 0,
             4: 1, 5: 1, 6: 1}
accident_clean['LIGHT_CONDITION_BINARY'] = accident_clean['LIGHT_CONDITION'].map(light_bin)

# Binary Speed Zone
light_bin = {30: 0, 40: 0, 50: 0, 60: 0,
             70: 1, 75: 1, 80: 1,
             90: 1, 100: 1, 110: 1}
accident_clean['SPEED_ZONE_BINARY'] = accident_clean['SPEED_ZONE'].map(light_bin)

# Vehicle Type Binary Remapping
veh_bin = { 1: 0, 2: 0,         # 2 and 4 wheelers light
           3: 1, 4: 1, 5: 1     # >4 heavy vehicle
           }
accident_clean.loc[:, "VEHICLE_TYPE_BINARY"] = accident_clean["VEHICLE_TYPE_CAT"].map(veh_bin)

*Making weather risk scoring based on road surface and atmospheric condition*

In [17]:
# Road Risk Scoring
surface_risk = { "Dry": 0, "Wet": 1,  "Muddy": 2, "Icy": 3, "Snowy": 3}
accident_clean.loc[:, "SURFACE_COND_RISK"] = accident_clean["SURFACE_COND_DESC"].map(surface_risk)

# Atmospheric Risk Scoring
atm_risk = { "Clear": 0, "Raining": 1, "Strong winds": 1,
    "Fog": 2, "Dust": 2, "Smoke": 2, "Snowing": 3}
accident_clean.loc[:, "ATMOSPH_COND_RISK"] = accident_clean["ATMOSPH_COND_DESC"].map(atm_risk)

# Weather Risk Scoring
accident_clean.loc[:, "WEATHER_RISK"] = \
    accident_clean["SURFACE_COND_RISK"] + accident_clean["ATMOSPH_COND_RISK"]

*Normalizing Continuous/Numeric Variable*

In [18]:
# Normalize Vehicle Age
accident_clean["VEHICLE_AGE_NORM"] = (
    accident_clean["VEHICLE_AGE"] - accident_clean["VEHICLE_AGE"].min()
) / (accident_clean["VEHICLE_AGE"].max() - accident_clean["VEHICLE_AGE"].min())

# Normalize Occupant
accident_clean["TOTAL_OCCUPANTS_NORM"] = (
    accident_clean["TOTAL_NO_OCCUPANTS"] - accident_clean["TOTAL_NO_OCCUPANTS"].min()
) / (accident_clean["TOTAL_NO_OCCUPANTS"].max() - accident_clean["TOTAL_NO_OCCUPANTS"].min())

*Counting Outlier from Normalized Features*

In [19]:
# Detecting Outlier
out_age = iqr_outliers(accident_clean["VEHICLE_AGE_NORM"])
print("Outlier counts in normalized vehicle age is:", out_age.count())

out_occupant = iqr_outliers(accident_clean["TOTAL_OCCUPANTS_NORM"])
print("Outlier counts in normalized total occupants is:", out_occupant.count())

Outlier counts in normalized vehicle age is: 1157
Outlier counts in normalized total occupants is: 2194


In [20]:
accident_clean.columns

Index(['ACCIDENT_NO', 'ACCIDENT_DATE', 'ACCIDENT_TIME', 'LIGHT_CONDITION',
       'ROAD_GEOMETRY', 'ROAD_GEOMETRY_DESC', 'SEVERITY', 'SPEED_ZONE',
       'ATMOSPH_COND', 'ATMOSPH_COND_DESC', 'SURFACE_COND',
       'SURFACE_COND_DESC', 'ACCIDENT_YEAR', 'VEHICLE_AGE',
       'TOTAL_NO_OCCUPANTS', 'VEHICLE_TYPE', 'TRAFFIC_CONTROL',
       'ROAD_SURFACE_TYPE', 'AGE_GROUP', 'SEATING_POSITION',
       'HELMET_BELT_WORN', 'SEVERITY_ORD', 'AGE_ORD', 'SEAT_CATEGORY',
       'VEHICLE_TYPE_CAT', 'TIME_OF_DAY', 'SEVERITY_BINARY',
       'SURFACE_COND_BINARY', 'ATMOSPH_COND_BINARY', 'SEAT_BINARY',
       'HELMET_BELT_BINARY', 'TRAFFIC_CONTROL_BINARY', 'ROAD_GEOMETRY_BINARY',
       'LIGHT_CONDITION_BINARY', 'SPEED_ZONE_BINARY', 'VEHICLE_TYPE_BINARY',
       'SURFACE_COND_RISK', 'ATMOSPH_COND_RISK', 'WEATHER_RISK',
       'VEHICLE_AGE_NORM', 'TOTAL_OCCUPANTS_NORM'],
      dtype='object')

In [21]:
accident_clean.isna().sum()

ACCIDENT_NO               0
ACCIDENT_DATE             0
ACCIDENT_TIME             0
LIGHT_CONDITION           0
ROAD_GEOMETRY             0
ROAD_GEOMETRY_DESC        0
SEVERITY                  0
SPEED_ZONE                0
ATMOSPH_COND              0
ATMOSPH_COND_DESC         0
SURFACE_COND              0
SURFACE_COND_DESC         0
ACCIDENT_YEAR             0
VEHICLE_AGE               0
TOTAL_NO_OCCUPANTS        0
VEHICLE_TYPE              0
TRAFFIC_CONTROL           0
ROAD_SURFACE_TYPE         0
AGE_GROUP                 0
SEATING_POSITION          0
HELMET_BELT_WORN          0
SEVERITY_ORD              0
AGE_ORD                   0
SEAT_CATEGORY             0
VEHICLE_TYPE_CAT          0
TIME_OF_DAY               0
SEVERITY_BINARY           0
SURFACE_COND_BINARY       0
ATMOSPH_COND_BINARY       0
SEAT_BINARY               0
HELMET_BELT_BINARY        0
TRAFFIC_CONTROL_BINARY    0
ROAD_GEOMETRY_BINARY      0
LIGHT_CONDITION_BINARY    0
SPEED_ZONE_BINARY         0
VEHICLE_TYPE_BINARY 

In [22]:
accident_clean.to_csv('C:/Users/Asus/Downloads/COMP20008/datasets/accident_clean.csv', index=False)

In [23]:
accident_clean.dtypes

ACCIDENT_NO                object
ACCIDENT_DATE              object
ACCIDENT_TIME              object
LIGHT_CONDITION             int64
ROAD_GEOMETRY               int64
ROAD_GEOMETRY_DESC         object
SEVERITY                    int64
SPEED_ZONE                  int64
ATMOSPH_COND                int64
ATMOSPH_COND_DESC          object
SURFACE_COND                int64
SURFACE_COND_DESC          object
ACCIDENT_YEAR               int32
VEHICLE_AGE               float64
TOTAL_NO_OCCUPANTS        float64
VEHICLE_TYPE              float64
TRAFFIC_CONTROL           float64
ROAD_SURFACE_TYPE         float64
AGE_GROUP                  object
SEATING_POSITION           object
HELMET_BELT_WORN          float64
SEVERITY_ORD                int64
AGE_ORD                     int64
SEAT_CATEGORY               int64
VEHICLE_TYPE_CAT            int64
TIME_OF_DAY                object
SEVERITY_BINARY             int64
SURFACE_COND_BINARY         int64
ATMOSPH_COND_BINARY         int64
SEAT_BINARY   